In [ ]:
# %pip install requests xarray netCDF4 pandas --quiet

'''
CELL 1
'''

from pathlib import Path
from datetime import date, timedelta
import time
import numpy as np
import pandas as pd
import requests
import xarray as xr

# Canada-ish bbox (adjust later if you want a smaller region)
BBOX_CANADA = dict(latitude_min=41.0, latitude_max=83.5,
                   longitude_min=-141.0, longitude_max=-52.5)

# Toronto bbox for testing
# Tiny & fast (≈2°×2°) — great for quick tests
BBOX_TORONTO_SMALL = dict(
    latitude_min  = 42.5,
    latitude_max  = 44.5,
    longitude_min = -80.5,
    longitude_max = -78.5,
)

# Moderate (≈5°×5°) — wider Southern Ontario
BBOX_TORONTO_MED = dict(
    latitude_min  = 41.0,
    latitude_max  = 46.0,
    longitude_min = -82.0,
    longitude_max = -77.0,
)

# Max single-call allowed (10°×10°) — still centered on Toronto
# (bigger = slower; this is the largest you should request in one go)
BBOX_TORONTO_MAX = dict(
    latitude_min  = 38.5,
    latitude_max  = 48.5,
    longitude_min = -84.5,
    longitude_max = -74.5,
)

def last_n_years_dates(n_years: int = 5):
    """
    Returns (start_yyyymmdd, end_yyyymmdd) covering the last n years,
    ending yesterday (UTC), using datetime.date throughout.
    """
    end = (date.today() - timedelta(days=1))
    # subtract years safely
    try:
        start = end.replace(year=end.year - n_years) + timedelta(days=1)
    except ValueError:
        # handle Feb 29 -> Mar 1 in non-leap years
        start = end.replace(year=end.year - n_years, month=3, day=1)
    return start.strftime("%Y%m%d"), end.strftime("%Y%m%d")

def tile_bbox_no_overlap(bbox: dict, max_span: float = 10.0, eps: float = 1e-6):
    """
    Split bbox into ≤max_span° tiles, using half-open intervals so neighbors don't overlap.
    Upper edge is exclusive (minus eps) except on the last tile in each direction.
    """
    lat_min, lat_max = bbox["latitude_min"], bbox["latitude_max"]
    lon_min, lon_max = bbox["longitude_min"], bbox["longitude_max"]

    lat_edges = list(np.arange(lat_min, lat_max, max_span)) + [lat_max]
    lon_edges = list(np.arange(lon_min, lon_max, max_span)) + [lon_max]

    for i in range(len(lat_edges) - 1):
        for j in range(len(lon_edges) - 1):
            up_lat = lat_edges[i+1] if i == len(lat_edges)-2 else lat_edges[i+1] - eps
            up_lon = lon_edges[j+1] if j == len(lon_edges)-2 else lon_edges[j+1] - eps
            yield dict(
                latitude_min=float(round(lat_edges[i], 6)),
                latitude_max=float(round(up_lat, 6)),
                longitude_min=float(round(lon_edges[j], 6)),
                longitude_max=float(round(up_lon, 6)),
            )

# Use small Toronto bbox for testing, then switch to Canada bbox
tiles = list(tile_bbox_no_overlap(BBOX_TORONTO_SMALL, max_span=10.0))

print(f"Total tiles: {len(tiles)}")


Total tiles: 1


In [ ]:
'''
CELL 2
'''

POWER_BASE = "https://power.larc.nasa.gov/api/temporal/daily/regional"
CACHE = Path("./power_cache"); CACHE.mkdir(parents=True, exist_ok=True)

def build_regional_url(param, bbox, start, end, fmt="NETCDF",
                       community="AG", time_standard="UTC"):
    return (
        f"{POWER_BASE}"
        f"?latitude-min={bbox['latitude_min']}&latitude-max={bbox['latitude_max']}"
        f"&longitude-min={bbox['longitude_min']}&longitude-max={bbox['longitude_max']}"
        f"&parameters={param}&community={community}"
        f"&time-standard={time_standard}&start={start}&end={end}&format={fmt}"
    )

def fetch_tile_year_netcdf(param: str, tile: dict, ystart: str, yend: str, pause: float = 0.4) -> Path:
    """
    Download one year's NetCDF for a single tile. POWER limits: ≤10° in lat/lon, 1 param/request.
    Uses a short pause between requests to be polite.
    """
    fname = (f"POWER_{param}_{ystart}_{yend}_"
             f"{tile['latitude_min']}_{tile['latitude_max']}_"
             f"{tile['longitude_min']}_{tile['longitude_max']}.nc")
    out = CACHE / fname
    if out.exists():
        return out

    url = build_regional_url(param, tile, ystart, yend, fmt="NETCDF")
    r = requests.get(url, stream=True, timeout=180)
    if r.status_code >= 400:
        # POWER returns details for 422; print for debugging
        try:
            print("API error payload:\n", r.text[:2000])
        except Exception:
            pass
        r.raise_for_status()

    ct = (r.headers.get("Content-Type","") or "").lower()
    if "netcdf" not in ct and "octet-stream" not in ct:
        raise RuntimeError(f"Expected NetCDF, got Content-Type={ct}. URL:\n{url}")

    with open(out, "wb") as f:
        for chunk in r.iter_content(1024*1024):
            if chunk:
                f.write(chunk)

    # small pause to avoid hammering the API
    time.sleep(pause)
    return out

def year_slices(start_yyyymmdd: str, end_yyyymmdd: str):
    """Yield (y_start, y_end) per year, clipped to the requested window."""
    start = pd.to_datetime(start_yyyymmdd, format="%Y%m%d").date()
    end   = pd.to_datetime(end_yyyymmdd,   format="%Y%m%d").date()
    for y in range(start.year, end.year + 1):
        ys = max(date(y, 1, 1), start)
        ye = min(date(y, 12, 31), end)
        yield ys.strftime("%Y%m%d"), ye.strftime("%Y%m%d")


In [ ]:
'''
CELL 3
'''

# Target variable for training: corrected daily precipitation (mm/day)
PARAM = "PRECTOTCORR"

# Last 5 years for testing; increase for real runs
start, end = last_n_years_dates(5)
print(f"Requesting {PARAM} for {start} → {end}")

# Download all tile-year files
nc_paths = []
for ys, ye in year_slices(start, end):
    print(f"\nYear chunk: {ys} → {ye}")
    for k, tile in enumerate(tiles, 1):
        print(f"  Tile {k}/{len(tiles)}: "
              f"lat[{tile['latitude_min']},{tile['latitude_max']}] "
              f"lon[{tile['longitude_min']},{tile['longitude_max']}] …")
        p = fetch_tile_year_netcdf(PARAM, tile, ys, ye)
        nc_paths.append(p)


Requesting PRECTOTCORR for 20201004 → 20251003

Year chunk: 20201004 → 20201231
  Tile 1/1: lat[42.5,44.5] lon[-80.5,-78.5] …

Year chunk: 20210101 → 20211231
  Tile 1/1: lat[42.5,44.5] lon[-80.5,-78.5] …

Year chunk: 20220101 → 20221231
  Tile 1/1: lat[42.5,44.5] lon[-80.5,-78.5] …

Year chunk: 20230101 → 20231231
  Tile 1/1: lat[42.5,44.5] lon[-80.5,-78.5] …

Year chunk: 20240101 → 20241231
  Tile 1/1: lat[42.5,44.5] lon[-80.5,-78.5] …

Year chunk: 20250101 → 20251003
  Tile 1/1: lat[42.5,44.5] lon[-80.5,-78.5] …


In [ ]:
'''
CELL 4
'''

import dask  # ensure it's importable
import xarray as xr

# pick the same engine you used earlier
# engine = "netcdf4" if "netcdf4" in xr.backends.list_engines() else "scipy"
engine = "scipy"

# Mosaic along coordinates (time, latitude, longitude)
# 'by_coords' merges the non-overlapping tiles; compat='override' avoids metadata conflicts.
ds = xr.open_mfdataset(
    [str(p) for p in nc_paths],
    engine=engine,
    combine="by_coords",
    data_vars="minimal",
    coords="minimal",
    compat="override",
    join="outer",
)

ds


<xarray.Dataset> Size: 234kB
Dimensions:      (time: 1826, lat: 5, lon: 3)
Coordinates:
  * time         (time) datetime64[ns] 15kB 2020-10-04 2020-10-05 ... 2025-10-03
  * lat          (lat) float64 40B 42.5 43.0 43.5 44.0 44.5
  * lon          (lon) float64 24B -80.0 -79.38 -78.75
Data variables:
    PRECTOTCORR  (time, lat, lon) float64 219kB dask.array<chunksize=(89, 5, 3), meta=np.ndarray>
Attributes: (12/22)
    acknowledgement:           The Prediction Of Worldwide Energy Resources (...
    comment:                   POWER data version 10.0.0 uses the source data...
    conventions:               CF-1.8, ACDD-1.3
    creator_email:             bradley.macpherson@nasa.gov
    creator_name:              Bradley Macpherson
    derived_from:              The data products are derived from data partne...
    ...                        ...
    publisher_name:            Paul Stackhouse
    references:                https://power.larc.nasa.gov
    source:                    Prediction Of Worldwide Energy Resource (POWER)
    standard_name_vocabulary:  CF Standard Name Table v72
    summary:                   The POWER project provides solar and meteorolo...
    version:                   POWER Version 10.0.0

In [ ]:
'''
CELL 5
'''

import pandas as pd

var = "PRECTOTCORR"
da  = ds[var]

# Robustly find coord names
def guess_lat_lon_coords(ds):
    for lat in ("lat","latitude","LAT","Latitude"):
        if lat in ds.coords: break
    else: raise ValueError(f"No lat coord found in {list(ds.coords)}")

    for lon in ("lon","longitude","LON","Longitude"):
        if lon in ds.coords: break
    else: raise ValueError(f"No lon coord found in {list(ds.coords)}")
    return lat, lon

lat_name, lon_name = guess_lat_lon_coords(ds)

# Summaries
print("Variable:", var)
print("Units:", da.attrs.get("units", ""))
print("Dims (mapping):", dict(da.sizes))  # <-- use sizes (not da.dims)
print("Time range:", pd.to_datetime(ds.time.values[0]).date(),
      "→", pd.to_datetime(ds.time.values[-1]).date())
print("Lat range:", float(ds[lat_name].min()), "→", float(ds[lat_name].max()))
print("Lon range:", float(ds[lon_name].min()), "→", float(ds[lon_name].max()))

# Tiny DataFrame sample (5 days × 10×10 cells)
sample = (
    da.isel({ "time": slice(0,5), lat_name: slice(0,10), lon_name: slice(0,10) })
      .to_dataframe()
      .reset_index()
)
print("\nSAMPLE DataFrame shape:", sample.shape)
sample.info(memory_usage="deep")
sample.head()


Variable: PRECTOTCORR
Units: mm/day
Dims (mapping): {'time': 1826, 'lat': 5, 'lon': 3}
Time range: 2020-10-04 → 2025-10-03
Lat range: 42.5 → 44.5
Lon range: -80.0 → -78.75

SAMPLE DataFrame shape: (75, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   time         75 non-null     datetime64[ns]
 1   lat          75 non-null     float64       
 2   lon          75 non-null     float64       
 3   PRECTOTCORR  75 non-null     float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 2.5 KB


,time,lat,lon,PRECTOTCORR
0,2020-10-04,42.5,-80.000,1.03
1,2020-10-04,42.5,-79.375,0.81
2,2020-10-04,42.5,-78.750,0.96
3,2020-10-04,43.0,-80.000,2.11
4,2020-10-04,43.0,-79.375,1.14


In [ ]:
'''
CELL 6
'''

import xarray as xr, pandas as pd, numpy as np
from pathlib import Path

# Choose extra variables to summarize as climatology features
EXTRA_PARAMS = [
  # Moisture & temp
  "RH2M",                     # Relative Humidity at 2 Meters (%)
  "T2M",                      # Temperature at 2 Meters (°C)
  "T2M_MAX",                  # Maximum Temperature at 2 Meters (°C)
  "T2M_MIN",                  # Minimum Temperature at 2 Meters (°C)
  "T2MDEW",                   # Dew Point Temperature at 2 Meters (°C)
  "QV2M",                     # Specific Humidity at 2 Meters (kg/kg)
  # Pressure
  "PS",                       # Surface Pressure (kPa)
  "SLP",                      # Sea Level Pressure (kPa)
  # Wind
  "WS10M",                    # Wind Speed at 10 Meters (m/s)
  # Radiation
  "ALLSKY_SFC_SW_DWN",        # All Sky Surface Shortwave Down
  # Water vapor column
  "PW",                       # Water Vapor Column (cm)
  # Extended parameters - could be enabled later if necessary
  # # Clouds
  # "CLOUD_AMT",                # Cloud Amount (tenths)
  # # Longwave
  # "ALLSKY_SFC_LW_DWN",        # All Sky Surface Longwave Down
  # # Soil wetness
  # "GWETTOP",                 # Top Layer Soil Wetness (unitless)
  # "GWETROOT",                # Root Zone Soil Wetness (unitless)
  # "GWETPROF",                # Profile Soil Wetness (unitless)
  # # Top-of-atmosphere solar / geometry
  # "TOA_SW_DWN",              # Top of Atmosphere Shortwave Down (W/m^2)
  # "SZA",                     # Solar Zenith Angle (degrees)
  # # Aerosols
  # "AOD_55",                  # Aerosol Optical Depth at 550 nm (unitless)
]

def ensure_unique_sorted_grid(ds: xr.Dataset, lat="lat", lon="lon") -> xr.Dataset:
    # Round to a common precision to kill fp jitter on borders
    ds = ds.assign_coords({lat: np.round(ds[lat].values.astype(float), 6),
                           lon: np.round(ds[lon].values.astype(float), 6)})
    # Keep first occurrence of each coordinate value
    lat_vals = pd.Index(ds[lat].values)
    lon_vals = pd.Index(ds[lon].values)
    ds = ds.isel({lat: ~lat_vals.duplicated(), lon: ~lon_vals.duplicated()})
    # Ensure sorted ascending
    return ds.sortby([lat, lon])

def daily_climatology_for_param(param: str, nc_paths_param: list[str]) -> xr.DataArray:
    engine = "netcdf4" if "netcdf4" in xr.backends.list_engines() else "scipy"
    ds_p = xr.open_mfdataset(
        [str(p) for p in nc_paths_param],
        engine=engine, combine="by_coords",
        data_vars="minimal", coords="minimal",
        compat="override", join="outer",
    )
    lat = "lat" if "lat" in ds_p.coords else "latitude"
    lon = "lon" if "lon" in ds_p.coords else "longitude"

    ds_p = ensure_unique_sorted_grid(ds_p, lat, lon)

    da = ds_p[param]
    # Drop Feb 29 → 365-day climatology
    da = da.sel(time=~((da.time.dt.month == 2) & (da.time.dt.day == 29)))
    clim = da.groupby("time.dayofyear").mean("time", keep_attrs=True)
    clim = clim.rename({"dayofyear": "doy"})
    clim.name = f"{param}_clim"
    return clim

# --- fetch files per param by reusing your tile/year loop ---
# We already have nc_paths for PRECTOTCORR. Let's build a helper to produce paths for other params.
def collect_paths_for_param(param: str, tiles, start: str, end: str):
    from pathlib import Path
    # your fetch_tile_year_netcdf(...) is already defined in previous cells
    paths = []
    for ys, ye in year_slices(start, end):
        for tile in tiles:
            paths.append(fetch_tile_year_netcdf(param, tile, ys, ye))
    return [str(p) for p in paths]

# Build a single Dataset of climatologies for all params
# For the Daily regional endpoint, requests are limited to one parameter per request
def build_climo_dataset(params, tiles, start, end) -> xr.Dataset:
    das = []
    for par in params:
        print(f"Computing climatology for {par} …")
        paths = collect_paths_for_param(par, tiles, start, end)
        da_clim = daily_climatology_for_param(par, paths)
        das.append(da_clim)
    ds_climo = xr.merge(das, compat="override", join="outer")
    return ds_climo

# Example usage with your existing variables: tiles, start, end (from earlier cells)
ds_climo = build_climo_dataset(EXTRA_PARAMS, tiles, start, end)
ds_climo


Computing climatology for T2M …
Computing climatology for T2M_MAX …
Computing climatology for T2M_MIN …
Computing climatology for RH2M …
Computing climatology for WS10M …
Computing climatology for PS …
Computing climatology for ALLSKY_SFC_SW_DWN …
Computing climatology for ALLSKY_SFC_LW_DWN …


<xarray.Dataset> Size: 706kB
Dimensions:                 (lat: 5, lon: 6, doy: 366)
Coordinates:
  * lat                     (lat) float64 40B 42.5 43.0 43.5 44.0 44.5
  * lon                     (lon) float64 48B -80.5 -80.0 -79.5 ... -78.75 -78.5
  * doy                     (doy) int64 3kB 1 2 3 4 5 6 ... 362 363 364 365 366
Data variables:
    T2M_clim                (doy, lat, lon) float64 88kB dask.array<chunksize=(1, 5, 3), meta=np.ndarray>
    T2M_MAX_clim            (doy, lat, lon) float64 88kB dask.array<chunksize=(1, 5, 3), meta=np.ndarray>
    T2M_MIN_clim            (doy, lat, lon) float64 88kB dask.array<chunksize=(1, 5, 3), meta=np.ndarray>
    RH2M_clim               (doy, lat, lon) float64 88kB dask.array<chunksize=(1, 5, 3), meta=np.ndarray>
    WS10M_clim              (doy, lat, lon) float64 88kB dask.array<chunksize=(1, 5, 3), meta=np.ndarray>
    PS_clim                 (doy, lat, lon) float64 88kB dask.array<chunksize=(1, 5, 3), meta=np.ndarray>
    ALLSKY_SFC_SW_DWN_clim  (doy, lat, lon) float64 88kB dask.array<chunksize=(1, 3, 3), meta=np.ndarray>
    ALLSKY_SFC_LW_DWN_clim  (doy, lat, lon) float64 88kB dask.array<chunksize=(1, 3, 3), meta=np.ndarray>
Attributes:
    base:                float32
    cell_methods:        time: mean
    definition:          The average air (dry bulb) temperature at 2 meters a...
    long_name:           Temperature at 2 Meters
    significant_digits:  2
    standard_name:       Temperature_at_2_Meters
    status:              official
    units:               C
    valid_max:           80.0
    valid_min:           -125.0

In [ ]:
'''
CELL 7
'''

import numpy as np
import pandas as pd
import xarray as xr

VAR = "PRECTOTCORR"  # target (mm/day)

# -------- prepare the daily target table --------
# remove Feb 29 so every year has 365 days
dss = ds[[VAR]].sel(time=~((ds.time.dt.month == 2) & (ds.time.dt.day == 29))).copy()

# xarray -> dataframe
df_y = dss.to_dataframe().reset_index()

# use your actual coord names:
lat_name = "lat" if "lat" in ds.coords else "latitude"
lon_name = "lon" if "lon" in ds.coords else "longitude"
df_y = df_y.rename(columns={lat_name: "lat", lon_name: "lon"})

# calendar features
df_y["year"] = df_y["time"].dt.year
df_y["doy"]  = df_y["time"].dt.dayofyear

# targets
df_y["rain_flag"] = (df_y[VAR] >= 0.1).astype("int8")
df_y["y_log1p"]   = np.log1p(df_y[VAR]).astype("float32")

# -------- bring in climatology features (depend only on doy, lat, lon) --------
# ds_climo was built earlier with columns like T2M_clim, RH2M_clim, ...
df_x = ds_climo.to_dataframe().reset_index()  # has columns: doy, lat/lon, <param>_clim

# merge keys: doy, lat, lon
data = df_y.merge(df_x, on=["doy", "lat", "lon"], how="left")

# cyclical encoding so the model "sees" seasonality smoothly
theta = 2 * np.pi * data["doy"] / 365.0
data["doy_sin"] = np.sin(theta).astype("float32")
data["doy_cos"] = np.cos(theta).astype("float32")

# memory-friendly dtypes
data["lat"] = data["lat"].astype("float32")
data["lon"] = data["lon"].astype("float32")
data[VAR]   = data[VAR].astype("float32")

# -------- inspect the complete dataset --------
print("Shape:", data.shape)
print("Columns:", list(data.columns)[:12], "…")
data.info(memory_usage="deep")
print("\nSample rows:")
display(data.sample(5, random_state=1))


Shape: (27375, 18)
Columns: ['time', 'lat', 'lon', 'PRECTOTCORR', 'year', 'doy', 'rain_flag', 'y_log1p', 'T2M_clim', 'T2M_MAX_clim', 'T2M_MIN_clim', 'RH2M_clim'] …
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27375 entries, 0 to 27374
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   time                    27375 non-null  datetime64[ns]
 1   lat                     27375 non-null  float32       
 2   lon                     27375 non-null  float32       
 3   PRECTOTCORR             27360 non-null  float32       
 4   year                    27375 non-null  int32         
 5   doy                     27375 non-null  int32         
 6   rain_flag               27375 non-null  int8          
 7   y_log1p                 27360 non-null  float32       
 8   T2M_clim                27375 non-null  float64       
 9   T2M_MAX_clim            27375 non-null  float64       
 10  T2

,time,lat,lon,PRECTOTCORR,year,doy,rain_flag,y_log1p,T2M_clim,T2M_MAX_clim,T2M_MIN_clim,RH2M_clim,WS10M_clim,PS_clim,ALLSKY_SFC_SW_DWN_clim,ALLSKY_SFC_LW_DWN_clim,doy_sin,doy_cos
2045,2021-02-17,43.0,-78.75,1.25,2021,48,1,0.810930,-3.144,-0.274,-6.364,86.232,6.306,98.816,NaN,NaN,0.735417,0.677615
23945,2025-02-17,43.0,-78.75,2.28,2025,48,1,1.187843,-3.144,-0.274,-6.364,86.232,6.306,98.816,NaN,NaN,0.735417,0.677615
7226,2022-01-28,44.0,-78.75,0.09,2022,28,0,0.086178,-6.328,-2.094,-10.862,85.844,7.188,99.120,NaN,NaN,0.463550,0.886071
1662,2021-01-22,44.5,-80.00,1.85,2021,22,1,1.047319,-7.978,-4.366,-11.442,92.336,5.962,98.652,NaN,NaN,0.369725,0.929141
2937,2021-04-17,44.5,-80.00,0.11,2021,107,1,0.104360,4.234,9.730,-0.056,77.062,3.826,98.248,NaN,NaN,0.963471,-0.267814


In [ ]:
'''
CELL 8
'''

# pick full years only (we removed Feb 29, so full year = 365 unique days)
days_per_year = data.groupby("year")["time"].nunique().sort_index()
full_years = days_per_year[days_per_year == 365].index.tolist()

assert len(full_years) >= 3, "Need at least 3 full years for train/val/test."

test_year = full_years[-1]
val_year  = full_years[-2]
train_years = full_years[:-2]

print("Train years:", train_years[:5], "…", train_years[-1] if len(train_years)>0 else None)
print("Val year  :", val_year)
print("Test year :", test_year)

train_df = data[data["year"].isin(train_years)].reset_index(drop=True)
val_df   = data[data["year"] == val_year].reset_index(drop=True)
test_df  = data[data["year"] == test_year].reset_index(drop=True)

for name, df_ in [("train", train_df), ("val", val_df), ("test", test_df)]:
    n_cells = df_[["lat","lon"]].drop_duplicates().shape[0]
    n_days  = df_["time"].nunique()
    print(f"{name:>5}: rows={len(df_):,}  cells={n_cells:,}  days={n_days}")


Train years: [2021, 2022] … 2022
Val year  : 2023
Test year : 2024
train: rows=10,950  cells=15  days=730
  val: rows=5,475  cells=15  days=365
 test: rows=5,475  cells=15  days=365


In [ ]:
'''
CELL 9
'''

# choose features/targets for modeling

# climatology feature columns created earlier end with "_clim"
climo_cols = [c for c in data.columns if c.endswith("_clim")]

X_cols = ["lat","lon","doy_sin","doy_cos"] + climo_cols
y_cls  = "rain_flag"
y_reg  = "y_log1p"

print("Feature columns:", X_cols[:8], "… total:", len(X_cols))


Feature columns: ['lat', 'lon', 'doy_sin', 'doy_cos', 'T2M_clim', 'T2M_MAX_clim', 'T2M_MIN_clim', 'RH2M_clim'] … total: 12


In [43]:
# CELL 10
# Quick check: any all-null climatology columns? (can happen if grids misalign)
climo_cols = [c for c in data.columns if c.endswith("_clim")]
empty_climos = [c for c in climo_cols if data[c].notna().sum() == 0]
print("Empty climatology cols:", empty_climos)


Empty climatology cols: ['ALLSKY_SFC_SW_DWN_clim', 'ALLSKY_SFC_LW_DWN_clim']


In [44]:
# CELL 11
from sklearn.model_selection import train_test_split

# features & targets
climo_cols = [c for c in data.columns if c.endswith("_clim")]
# drop all-null climo columns so we don't carry useless features
empty_climos = [c for c in climo_cols if data[c].notna().sum() == 0]
if empty_climos:
    print("Dropping all-null climo cols:", empty_climos)
climo_cols = [c for c in climo_cols if c not in empty_climos]

X_cols = ["lat","lon","doy_sin","doy_cos"] + climo_cols
y_cls  = "rain_flag"   # stage 1 target
y_reg  = "y_log1p"     # stage 2 target (on rainy rows)

def XY(df):
    return df[X_cols], df[y_cls].astype(int)

def XR(df):
    mask = df[y_cls] == 1
    return df.loc[mask, X_cols], df.loc[mask, y_reg]

Xtr, ytr = XY(train_df)
Xva, yva = XY(val_df)
Xte, yte = XY(test_df)

Xtr_rain, ytr_rain = XR(train_df)
Xva_rain, yva_rain = XR(val_df)
Xte_rain, yte_rain = XR(test_df)

print("Features:", len(X_cols), " | Train rows:", len(train_df), " | Rain-only rows:", len(Xtr_rain))


Dropping all-null climo cols: ['ALLSKY_SFC_SW_DWN_clim', 'ALLSKY_SFC_LW_DWN_clim']
Features: 10  | Train rows: 10950  | Rain-only rows: 8763


In [45]:
# CELL 12
# %pip install scikit-learn --quiet
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, brier_score_loss

clf = Pipeline([
    ("imp",   SimpleImputer(strategy="median")),
    ("scal",  StandardScaler(with_mean=False)),
    ("logit", LogisticRegression(max_iter=1000, class_weight="balanced")),
])

clf.fit(Xtr, ytr)

# validation
p_va = clf.predict_proba(Xva)[:,1]
print("Classifier — Val AUC:", round(roc_auc_score(yva, p_va), 3))
print("Classifier — Val Brier:", round(brier_score_loss(yva, p_va), 4))

# test
p_te = clf.predict_proba(Xte)[:,1]
print("Classifier — Test AUC:", round(roc_auc_score(yte, p_te), 3))
print("Classifier — Test Brier:", round(brier_score_loss(yte, p_te), 4))


Classifier — Val AUC: 0.662
Classifier — Val Brier: 0.23
Classifier — Test AUC: 0.673
Classifier — Test Brier: 0.2296


In [46]:
# CELL 13
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

reg = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("rf",  RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
])

reg.fit(Xtr_rain, ytr_rain)

pred_va_log = reg.predict(Xva_rain)
pred_te_log = reg.predict(Xte_rain)

def rmse(y_true, y_pred):
    # Compat with older scikit-learn (no squared= kw)
    return np.sqrt(mean_squared_error(y_true, y_pred))

mae_va = mean_absolute_error(np.expm1(yva_rain), np.expm1(pred_va_log))
rmse_va = rmse(np.expm1(yva_rain), np.expm1(pred_va_log))
mae_te = mean_absolute_error(np.expm1(yte_rain), np.expm1(pred_te_log))
rmse_te = rmse(np.expm1(yte_rain), np.expm1(pred_te_log))

print(f"Regressor (rain-only) — Val MAE: {mae_va:.3f} mm, RMSE: {rmse_va:.3f} mm")
print(f"Regressor (rain-only) — Test MAE: {mae_te:.3f} mm, RMSE: {rmse_te:.3f} mm")

Regressor (rain-only) — Val MAE: 2.358 mm, RMSE: 3.929 mm
Regressor (rain-only) — Test MAE: 2.129 mm, RMSE: 3.637 mm


In [47]:
# CELL 14
from sklearn.metrics import mean_absolute_error, mean_squared_error

def combined_expected_mm(df):
    # Expected precip = p_rain * amount_if_rain
    p = clf.predict_proba(df[X_cols])[:,1]
    amt_log = reg.predict(df[X_cols])                # model can score all rows
    amt = np.expm1(amt_log).clip(min=0)
    return p * amt

# Climatology baseline from TRAIN years
climo_baseline = (
    train_df.groupby(["lat","lon","doy"])
            .agg(p_rain=("rain_flag","mean"),
                 med_mm=("PRECTOTCORR", lambda s: s[s>0].median()))
            .reset_index()
)
climo_baseline["med_mm"] = climo_baseline["med_mm"].fillna(0.0)

def baseline_expected_mm(df):
    key = df[["lat","lon","doy"]]
    base = key.merge(climo_baseline, on=["lat","lon","doy"], how="left").fillna(0.0)
    return base["p_rain"].to_numpy() * base["med_mm"].to_numpy()

# Evaluate on TEST
y_true = test_df["PRECTOTCORR"].to_numpy()
y_model = combined_expected_mm(test_df)
y_base  = baseline_expected_mm(test_df)

def report(name, yhat):
    mae = mean_absolute_error(y_true, yhat)
    r   = rmse(y_true, yhat)
    print(f"{name:20s}  MAE={mae:.3f} mm   RMSE={r:.3f} mm")

print("\n== Test (all days) ==")
report("Model (2-stage)", y_model)
report("Baseline (climo)", y_base)



== Test (all days) ==
Model (2-stage)       MAE=1.773 mm   RMSE=3.306 mm
Baseline (climo)      MAE=2.172 mm   RMSE=3.661 mm


In [ ]:
# CELL 15
import numpy as np
import pandas as pd

def predict_precip_mm(lat, lon, date_str):
    """
    Predicts expected precipitation (mm) for a date/location.
    Returns dict(p_rain, amount_if_rain_mm, expected_mm).
    """
    dt = pd.to_datetime(date_str)
    doy = int(dt.strftime("%j"))  # 1..365 (we removed Feb 29 in training)

    row = {
        "lat": float(lat),
        "lon": float(lon),
        "doy": doy,
        "doy_sin": np.sin(2*np.pi*doy/365.0),
        "doy_cos": np.cos(2*np.pi*doy/365.0),
    }

    # pull climatology features at exact (lat, lon, doy) used for training
    # snap to nearest existing grid point to avoid missed joins
    lat_g = float(data.loc[(data["lat"]-lat).abs().idxmin(), "lat"])
    lon_g = float(data.loc[(data["lon"]-lon).abs().idxmin(), "lon"])
    key = {"lat": lat_g, "lon": lon_g, "doy": doy}

    # fetch climo values from a tiny frame (fast)
    cl = (ds_climo.to_dataframe().reset_index()
            .query("lat == @lat_g and lon == @lon_g and doy == @doy"))
    for c in [c for c in cl.columns if c.endswith("_clim")]:
        row[c] = float(cl[c].iloc[0]) if not cl.empty else 0.0

    X = pd.DataFrame([row])[X_cols]
    p = float(clf.predict_proba(X)[:,1])
    amt = float(np.expm1(reg.predict(X)))
    return {"p_rain": p, "amount_if_rain_mm": max(0.0, amt), "expected_mm": max(0.0, p*amt)}


In [49]:
# Example:
predict_precip_mm(43.65, -79.38, "2025-11-15")

/tmp/ipykernel_15599/1401866120.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p = float(clf.predict_proba(X)[:,1])
/tmp/ipykernel_15599/1401866120.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  amt = float(np.expm1(reg.predict(X)))


{'p_rain': 0.4675618716240334,
 'amount_if_rain_mm': 1.3175316935793382,
 'expected_mm': 0.6160275845739378}